# Análisis y preprocesamiento de los datos

Se explorarán, en pequeños experimentos, distintas formas de representación de los datos del corpus WiNER (Ghaddar y Langlais 2017) para utilizarlos en la tarea de reconocimiento de entidades nombradas. Para esto se exploran distintas combinaciones de vectores de palabras como representación de una instancia de entrenamiento (Iacobacci et al. 2016).

## Descripción del Corpus WiNER

* Documents.tar.bz2 : este archivo contiene 3239540 artículos de Wikipedia repartidos en 3223 archivos. Cada archivo contiene aproximadamente 1000 artículos nombrados por sus respectivos IDs. Los artículos están indexados por su wikiID seguidos de oraciones (una por línea), donde las palabras son remplazadas por sus ids.

      ID <number>
      1234 4522 23 4 4567
      456 21 9890 123 7 0

* document.vocab : este archivo contiene el mapeo de palabras (case sensitive); el formato es: 
 
      palabra #ocurrencias
    
  El ID de cada palabra es el número de línea en la cual ocurre.

In [1]:
import pandas as pd
import numpy as np
import time
import gensim
from gensim.models import Word2Vec, KeyedVectors

### Cargamos los datos

In [2]:
word_mapping = pd.read_csv('./corpus_WiNER/document.vocab', sep=' ', header=None, 
                           names=['word', 'frequency'], keep_default_na=False) 
                        # con esto evito que el parser trate como nan value a algunas palabras.

In [3]:
print(word_mapping.shape)
word_mapping.head()

(6931358, 2)


,word,frequency
0,the,73411953
1,",",68044881
2,.,54241850
3,of,40550466
4,and,34602894


In [4]:
len(word_mapping['word'].unique())

6931358

### Cargamos los artículos del Documento "0"

In [5]:
doc_0 = pd.read_csv('./corpus_WiNER/Documents/0', sep='ID', engine='python', header=None, names=['sentence', 'art_ID'])

In [6]:
doc_0.shape

(130209, 2)

In [7]:
doc_0.head()

,sentence,art_ID
0,NaN,431388.0
1,650 5590 753942 12 189 243 1 2039 47 257 682 1...,NaN
2,34 23207 1523 4 57193 15 10777 14353 4 157019 ...,NaN
3,418 0 1858 101 0 1322 1 5590 18 860 729 14 92 ...,NaN
4,650 5590 1523 8 1136 15 11301 575 1156 1 2746 ...,NaN


Vamos a asociarle a cada oración el ID del artículo al cual pertenece.

Es importante recordar que el orden de las oraciones está dado por los índices del dataframe.

In [8]:
art_ID_list = doc_0['art_ID'].tolist()
art_ID = 0
for idx, elem in enumerate(art_ID_list):
    if not np.isnan(elem):
        art_ID = elem
    else:
        art_ID_list[idx] = art_ID
doc_0['art_ID'] = art_ID_list

Removemos ahora las filas con NaN que contenian los ID de los artículos inicialmente.

In [9]:
doc_0 = doc_0.dropna()

In [10]:
print('El documento contiene {} oraciones.'.format(doc_0.shape[0]))
print('El documento contiene {} artículos'.format(len(doc_0['art_ID'].unique())))

El documento contiene 128395 oraciones.
El documento contiene 1814 artículos


Hacemos que cada sentencia sea una lista de palabras codificadas y casteamos a int

In [11]:
doc_0['sentence'] = doc_0['sentence'].map(lambda x: list(map(int, x.split(' '))))

In [12]:
doc_0.head()

,sentence,art_ID
1,"[650, 5590, 753942, 12, 189, 243, 1, 2039, 47,...",431388.0
2,"[34, 23207, 1523, 4, 57193, 15, 10777, 14353, ...",431388.0
3,"[418, 0, 1858, 101, 0, 1322, 1, 5590, 18, 860,...",431388.0
4,"[650, 5590, 1523, 8, 1136, 15, 11301, 575, 115...",431388.0
5,"[152, 642, 16, 8143, 23122, 20, 0, 662955, 16,...",431388.0


Nos quedamos con un artículo

In [13]:
article = doc_0[doc_0.art_ID == 1000]
article.head()

,sentence,art_ID
2756,"[63903, 24730, 9, 7, 3035, 4104, 7584, 1, 355,...",1000.0
2757,"[3654, 16, 2502, 39413, 1, 24730, 9, 44, 3, 99...",1000.0
2758,"[24730, 35, 49, 3521, 15, 575, 1, 15, 2440, 1,...",1000.0
2759,"[152, 112, 8, 2037, 20, 52, 54, 3035, 17572, 3...",1000.0
2760,"[64, 62, 5787, 1132, 15, 0, 144, 24730, 1312, ...",1000.0


Reconstruimos la primera oración del artículo

In [14]:
def sentence_decoder(sentence, word_mapping):
    dec_sentence = []
    for idx in sentence:
        mapped_w = word_mapping.loc[idx, 'word']
        dec_sentence.append(mapped_w)
    return dec_sentence 

In [15]:
# Escribimos article.columns.get_loc('sentence') para evitar hardcodear el índice correspondiente
# a la columna 'sentence' que en este caso es 0
sentence = article.iloc[0, article.columns.get_loc('sentence')]

In [16]:
dec_sentence = sentence_decoder(sentence, word_mapping)
' '.join(dec_sentence)

'Hercule Poirot is a fictional Belgian detective , created by Agatha Christie .'

### Word embeddings utilizando el modelo pre-entrenado word2vec de Google

Utilizaremos la librería Gensim https://radimrehurek.com/gensim/

Cargamos Google's pre-trained Word2Vec model.

Utilizando KeyedVectors para cargar el modelo tiene la desventaja de que no se puede seguir entrenando. Pero es más eficiente que utilizar gensim.models.Word2Vec
https://radimrehurek.com/gensim/models/keyedvectors.html#module-gensim.models.keyedvectors

In [17]:
start = time.time()
# model = KeyedVectors.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin', binary=True)
# model.save('./models/word2vecGoogle.model')
model = KeyedVectors.load('./models/word2vecGoogle.model')
end = time.time()
print('demora: {}'.format(end-start))
# model = Word2Vec.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin', binary=True)

demora: 12.730851888656616


In [18]:
print('Cantidad de word embeddings: {}'.format(len(model.vectors)))

Cantidad de word embeddings: 3000000


In [19]:
print('Dimensionalidad de los vectores: {}'.format(model.vector_size))

Dimensionalidad de los vectores: 300


## Exploremos distintas combinaciones de vectores de palabras

### Concatenación

Este método consiste en concatenar los vectores de palabras que rodean una palabra objetivo en un vector más grande, que tiene un tamaño igual a las dimensiones agregadas de todos las proyecciones (embeddings) individuales.

- $w_{ij}$ = peso asociado con la i-ésima dimensión del vector de la j-ésima palabra en la oración. NOTA: con los vectores de palabras de una oración se forma una matriz $w^{\space D\space x\space L}$ donde $L$ es la cantidad de palabras de esa oración.
- $D$ = dimensionalidad de los word vectors originales. Por ejemplo, al usar el modelo word2vec de Google sabemos que D = 300.
- $W$ = tamaño de ventana que se define como el número de palabras en un solo lado.

Nos interesa representar el contexto de la I-ésima palabra de la oración. 

La i-ésima dimensión del vector de concatenación, que tiene un tamaño de $2 W D$, se calcula de la siguiente manera:

$$ e_{i} =\begin{cases} 
      w_{i\space mod \space D,\space\space I \space - \space W \space + \space \left\lfloor{\frac{i}{D}}\right\rfloor} & \left\lfloor{\frac{i}{D}}\right\rfloor < W \\
      w_{i\space mod \space D,\space\space I \space - \space W \space + \space 1\space  +\space\left\lfloor{\frac{i}{D}}\right\rfloor} & c.c.
   \end{cases}$$

In [20]:
# TODO: code me!

### Promedio

Como su nombre indica, se calcula el centroide de los embeddings de todas las palabras circundantes. La fórmula divide cada dimensión en $2W$ ya que el número de palabras del contexto es dos veces el tamaño de la ventana:

$$\sum_{\substack{j\space=\space I-W \\ j\space\neq\space I}}^{I + W} \frac{w_{ij}}{2W}$$

In [21]:
# TODO: code me!

### Decaimiento fraccional

Una tercera estrategia para construir un vector de carácteristicas en base a los embeddings de palabras contextuales está inspirada en la forma en que Word2vec combina las palabras en el contexto. Aquí, se supone que la importancia de una palabra para nuestra representación es inversamente proporcional a su distancia respecto a la palabra objetivo.

Por lo tanto, las palabras contextuales se ponderan en función de su distancia de la palabra objetivo:

$$\sum_{\substack{j\space=\space I-W \\ j\space\neq\space I}}^{I + W} w_{ij} *\frac{W - \lvert I-j\rvert}{W}$$


In [22]:
# TODO: code me!

### Decaimiento exponencial

Funciona de manera similar al decaimiento fraccional, que le da más importancia al contexto cercano, pero en este caso la ponderación se realiza exponencialmente:

$$\sum_{\substack{j\space=\space I-W \\ j\space\neq\space I}}^{I + W} w_{ij} * (1 - \alpha)^{\lvert \space I\space-\space j\space\rvert\space-\space1}$$

donde $\alpha = 1 - 0.1^{(W-1)^{-1}}$ es el parámetro de decaimiento. Elegimos el parámetro de tal manera que las palabras inmediatas que rodean a la palabra objetivo contribuyen 10 veces más que las últimas palabras en ambos lados de la ventana.

In [23]:
# TODO: code me!

TODO: cambiar redacción y fórmulas de las distintas estrategias si se decide incluir al vector de la palabra objetivo en las operaciones que generan el nuevo vector.

Agregar también que para el caso de concatenación se va a realizar padding de 0's a izquierda o derecha cuando corresponda.

## Exploremos CoarseNE.tar.bz2

Contiene menciones anotadas automáticamente con etiquetas de entidades nombradas (PER, LOC, ORG y MISC).

El formato es:

    ID artID
    sentIdx begin end entityType
    
donde entityType[0] = PER | entityType[1] = LOC | entityType[2] = ORG | entityType[3] = MISC

In [24]:
coarseNE_0 = pd.read_csv('./corpus_WiNER/CoarseNE/0', sep='ID', engine='python', header=None, names=['named-entity', 'art_ID'])

In [25]:
print(coarseNE_0.shape)
coarseNE_0.head()

(259470, 2)


,named-entity,art_ID
0,NaN,1000.0
1,0\t0\t2\t0,NaN
2,0\t5\t6\t1,NaN
3,0\t10\t12\t0,NaN
4,1\t2\t4\t0,NaN


Aplicamos el mismo truco que utilizamos en los documentos para propagar los art_ID según corresponda

In [26]:
art_ID_list = coarseNE_0['art_ID'].tolist()
art_ID = 0
for idx, elem in enumerate(art_ID_list):
    if not np.isnan(elem):
        art_ID = elem
    else:
        art_ID_list[idx] = art_ID
coarseNE_0['art_ID'] = art_ID_list
coarseNE_0 = coarseNE_0.dropna()

In [27]:
print('coarseNE_0 contiene {} entidades.'.format(coarseNE_0.shape[0]))
print('coarseNE_0 contiene {} artículos'.format(len(coarseNE_0['art_ID'].unique())))

coarseNE_0 contiene 257656 entidades.
coarseNE_0 contiene 1810 artículos


No todos los artículos que ocurren en Documents/0 se encuentran en CoarseNE/0

TODO: chequear si esto sucede porque esos artículos no contienen entidades nombradas.

In [29]:
print('Artículos que están presentes en Documents/0 pero no en CoarseNE/0: {}'
      .format(list(set(doc_0.art_ID.unique()) - set(coarseNE_0.art_ID.unique()))))

Artículos que están presentes en Documents/0 pero no en CoarseNE/0: [431177.0, 432375.0, 432318.0, 10263.0]


Creamos nuevas columnas con la info de la columna named-entity para mejor manipulación

In [30]:
def get_tag(x):
    tags = ['PER', 'LOC', 'ORG', 'MISC']
    return tags[x]

In [31]:
coarseNE_0['named-entity'] = coarseNE_0['named-entity'].map(lambda x: x.split('\t'))
coarseNE_0['senIdx'] = coarseNE_0['named-entity'].map(lambda x: int(x[0]))
coarseNE_0['begin'] = coarseNE_0['named-entity'].map(lambda x: int(x[1]))
coarseNE_0['end'] = coarseNE_0['named-entity'].map(lambda x: int(x[2]))
coarseNE_0['entityType'] = coarseNE_0['named-entity'].map(lambda x: get_tag(int(x[3])))
coarseNE_0 = coarseNE_0.drop(columns='named-entity')

In [32]:
coarseNE_0.head()

,art_ID,senIdx,begin,end,entityType
1,1000.0,0,0,2,PER
2,1000.0,0,5,6,LOC
3,1000.0,0,10,12,PER
4,1000.0,1,2,4,PER
5,1000.0,1,5,6,PER


### Veamos como están anotadas las entidades nombradas de una oración en particular 

In [33]:
article = doc_0[doc_0.art_ID == 1000]
sentence = article.iloc[0, article.columns.get_loc('sentence')]
dec_sentence = sentence_decoder(sentence, word_mapping)
' '.join(dec_sentence)

'Hercule Poirot is a fictional Belgian detective , created by Agatha Christie .'

In [34]:
art_entities = coarseNE_0[coarseNE_0.art_ID == 1000]
entities_sen_0 = art_entities[art_entities.senIdx == 0]
entities_sen_0

,art_ID,senIdx,begin,end,entityType
1,1000.0,0,0,2,PER
2,1000.0,0,5,6,LOC
3,1000.0,0,10,12,PER


In [35]:
for idx, row in entities_sen_0.iterrows():
    print('{} : {}'.format(' '.join(dec_sentence[row['begin']:row['end']]), row['entityType']))

Hercule Poirot : PER
Belgian : LOC
Agatha Christie : PER


Preguntas:

- En las cuatro estrategias que plantea el paper, el vector de contexto que se genera no considera la información de la palabra objetivo (i.e. no se tiene en cuenta el vector objetivo para generar este nuevo vector). Sería mejor incluir el vector de la palabra objetivo para que la representación sea más informativa?

<br>
- Los experimentos para medir cuales representaciones funcionan mejor recién se van a poder realizar cuando tengamos la implementación de la red, verdad?

<br>
- Charlar sobre diseño del cargado y manipulación del Corpus. (mapeos, futura partición de datos para entrenamiento supervisado y no supervisado, etc)

NOTA: 
      #articulos de CoarseNE/0 < #articulos de Documents/0
      #articulos de CoarseNE/1 == #articulos de Documents/1 (y son los mismos)
      
      Tanto el dir CoarseNE como Documents contienen 3223 archivos
      
Es raro que el archivo 0 de ambos difiera en 4 la cantidad de articulos que posee.
Sera porque justo esos 4 articulos no poseen ninguna entidad nombrada?
TODO: chequear esto

In [36]:
unpickled_df = pd.read_pickle("./doc_77")

In [37]:
unpickled_df.head()

,sentence,art_ID
1,"[Belton, House, is, a, Grade, I, listed, count...",145492.0
2,"[The, mansion, is, surrounded, by, formal, gar...",145492.0
3,"[Belton, has, been, described, as, a, compilat...",145492.0
4,"[The, house, has, also, been, described, as, t...",145492.0
5,"[Only, Brympton, d'Evercy, has, been, similarl...",145492.0


In [38]:
' '.join(unpickled_df.loc[1, 'sentence'])

'Belton House is a Grade I listed country house in Belton near Grantham , Lincolnshire , England .'

Cantidad de oraciones: 54607542
Cantidad de artículos: 3239540